# Estimate Binomial draw probabilility

In [1]:
using DynamicHMCModels

Define a structure to hold the data.

In [2]:
struct BernoulliProblem
    "Total number of draws in the data."
    n::Int
    "Number of draws `==1` in the data"
    s::Vector{Int}
end;

Make the type callable with the parameters *as a single argument*.

In [3]:
function (problem::BernoulliProblem)((α, )::NamedTuple{(:α, )})
    @unpack n, s = problem        # extract the data
    loglikelihood(Binomial(n, α), s)
end

Create the data and complete setting up the problem.

In [4]:
obs = rand(Binomial(9, 2/3), 1)
p = BernoulliProblem(9, obs)
p((α = 0.5, ))

-1.4020427180880297

Write a function to return properly dimensioned transformation.

In [5]:
problem_transformation(p::BernoulliProblem) =
    as((α = as𝕀, ),  )

problem_transformation (generic function with 1 method)

Use a flat priors (the default, omitted) for α

In [6]:
P = TransformedLogDensity(problem_transformation(p), p)
∇P = LogDensityRejectErrors(ADgradient(:ForwardDiff, P));

Sample

In [7]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000)

MCMC, adapting ϵ (75 steps)
3.1e-5 s/step ...done
MCMC, adapting ϵ (25 steps)
4.0e-5 s/step ...done
MCMC, adapting ϵ (50 steps)
3.6e-5 s/step ...done
MCMC, adapting ϵ (100 steps)
3.5e-5 s/step ...done
MCMC, adapting ϵ (200 steps)
2.9e-5 s/step ...done
MCMC, adapting ϵ (400 steps)
3.2e-5 s/step ...done
MCMC, adapting ϵ (50 steps)
4.0e-5 s/step ...done
MCMC (1000 steps)
2.9e-5 s/step ...done


(NUTS_Transition{Array{Float64,1},Float64}[NUTS_Transition{Array{Float64,1},Float64}([0.49147520986245274], -2.9790724121904377, 2, DynamicHMC.DoubledTurn, 0.9579148706861392, 3), NUTS_Transition{Array{Float64,1},Float64}([0.7821672702144686], -3.2231931459540153, 1, DynamicHMC.DoubledTurn, 0.9033589455203096, 1), NUTS_Transition{Array{Float64,1},Float64}([1.4390088619285764], -4.700143576479869, 1, DynamicHMC.DoubledTurn, 0.6891903970150741, 1), NUTS_Transition{Array{Float64,1},Float64}([0.711029806772328], -4.293120489707723, 1, DynamicHMC.AdjacentTurn, 1.0, 3), NUTS_Transition{Array{Float64,1},Float64}([1.2943339569363306], -4.300926698659309, 1, DynamicHMC.DoubledTurn, 0.7330433910571466, 1), NUTS_Transition{Array{Float64,1},Float64}([0.6549074563995683], -5.144456000047516, 2, DynamicHMC.DoubledTurn, 0.9193133070007939, 3), NUTS_Transition{Array{Float64,1},Float64}([1.181895734366327], -5.232907215687822, 2, DynamicHMC.DoubledTurn, 0.7471228265981934, 3), NUTS_Transition{Array{Flo

To get the posterior for ``α`` use `get_position` and then transform back.

In [8]:
posterior = TransformVariables.transform.(Ref(problem_transformation(p)), get_position.(chain));

Extract the parameter.

In [9]:
posterior_α = first.(posterior);

check the effective sample size

In [10]:
ess_α = effective_sample_size(posterior_α)

377.3502855000167

NUTS-specific statistics

In [11]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.91, min/25%/median/75%/max: 0.1 0.88 0.97 1.0 1.0
  termination: AdjacentTurn => 27% DoubledTurn => 73%
  depth: 1 => 58% 2 => 42%


check the mean

In [12]:
mean(posterior_α)

0.5450459122230754

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*